In [8]:
from pathlib import Path
repo_path = Path('/home/ricardo/ABUS2023_documents/tdsc_abus23')

In [11]:
import numpy as np
import SimpleITK as sitk
import pandas as pd
from torchvision.transforms import (
    Compose,
    Resize,
    InterpolationMode,
)
from PIL import Image
from tqdm import tqdm

In [12]:
# Expansion HP
x_expansion = 865
y_expansion = 865
x_resizing = 512
y_resizing = 512
file_format = 'mha'
folder_name = f'full-slice_{x_resizing}x{y_resizing}_all'
data_directory = repo_path / 'data/challange_2023' / 'Test'
data_directory.mkdir(exist_ok=True, parents=True)

# transforms
preprocess_im = Compose(
        [
            Resize((x_resizing, y_resizing), interpolation= InterpolationMode.BILINEAR),
        ]
)

# new images and labels
save_dir = data_directory / folder_name
save_dir.mkdir(exist_ok=True)
im_dir = save_dir / f'image_{file_format}'
im_dir.mkdir(exist_ok=True)

### create metadata
# raw data path
raw_data = repo_path / 'input'
files = [f for f in raw_data.glob('**/*') if f.suffix == '.nrrd']
iter = tqdm(files, total=len(files))
metadata = pd.DataFrame(columns=['case_id', 'data_path', 'shape'])

for i, im_path in enumerate(iter):
    # get 3D image
    im_sitk = sitk.ReadImage(im_path)
    # store metadata
    metadata.loc[i, 'case_id'] = im_path.stem.split('_')[-1]
    metadata.loc[i, 'data_path'] = 'DATA/'+im_path.name
    metadata.loc[i, 'shape'] = im_sitk.GetSize()
# save metadata
metadata = metadata.sort_values(by=['case_id'])
metadata.to_csv(data_directory / 'metadata.csv', index=False)

100%|██████████| 30/30 [00:39<00:00,  1.31s/it]
